In [133]:
# Imports
import os
import pandas as pd
import sqlite3

# No scientific notation, only ints
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [134]:
# Creates dataframes for each park's animal data

# Creates a dictionary from park 4 letter code to a dataframe of it's animal data.
park_frames = {}
biodiversity_data_path = "../data/park_biodiversity_data/"
for file in os.listdir(biodiversity_data_path):
    tag = file.split(".")[0]
    park_frames[tag] = pd.read_csv(biodiversity_data_path + file)

# Create dataframe for visitation data and location data.
visitation_frame = pd.read_csv("../data/annual_visitation.csv")
location_frame = pd.read_csv("../data/location_data.csv")

In [135]:
# Print columns of each dataframe
print(park_frames['NOCA'].columns)
print(visitation_frame.columns)
print(location_frame.columns)

Index(['Park Code', 'Park Name', 'Category', 'Category Sort', 'Order',
       'Family', 'Taxon Code', 'TSN', 'Taxon Record Status', 'Scientific Name',
       'Common Names', 'Synonyms', 'Occurrence'],
      dtype='object')
Index(['Region_Name', 'Park_Name', 'Park_Type', 'Year', 'TRV'], dtype='object')
Index(['Park_Code', 'Park_Name', 'State', 'Acres', 'Latitude', 'Longitude'], dtype='object')


In [136]:
# Joins all the park dataframes into one large dataframe and nicely names columns
wildlife_frame = pd.concat([x for x in park_frames.values()], axis=0)
wildlife_frame.columns = wildlife_frame.columns.str.replace(' ', '_')
visitation_frame.columns = visitation_frame.columns.str.replace('TRV', 'Visitors')

In [137]:
#Filters the data

# Removes unnecessary columns
wildlife_frame = wildlife_frame.drop(["Taxon_Code", "TSN", "Synonyms"], axis=1) 

# Removes rows that aren't actively being seen in the park
wildlife_frame = wildlife_frame.query("Occurrence == 'Present'") 
wildlife_frame = wildlife_frame.query("Taxon_Record_Status == 'Active'")

# Location frame standardization
location_frame["Park_Name"] = location_frame["Park_Name"].str.replace('Of The', "of the")

# Visitation frame name standardization.
visitation_frame["Park_Name"] = visitation_frame["Park_Name"].str.replace('Kings Canyon NP', "Sequoia and Kings Canyon NPs")
visitation_frame["Park_Name"] = visitation_frame["Park_Name"].str.replace('Sequoia N', "Sequoia and Kings Canyon NPs")
visitation_frame["Park_Name"] = visitation_frame["Park_Name"].str.replace('Wrangell-St.', "Wrangell - St")
visitation_frame["Park_Name"] = visitation_frame["Park_Name"].str.replace('NP', "National Park")
visitation_frame["Park_Name"] = visitation_frame["Park_Name"].str.replace('& PRES', "and Preserve")

# Removes all rows > 10 years old for visitation data and then merges them taking the average.
visitation_frame = visitation_frame.query("Year >= 2012")
visitation_frame = visitation_frame.drop(['Year'], axis=1)
visitation_frame["Visitors"] = visitation_frame["Visitors"].str.replace(',', '')
visitation_frame["Visitors"] = pd.to_numeric(visitation_frame["Visitors"])
visitation_frame = visitation_frame.groupby(["Park_Name"]).mean().round(0)

In [138]:
# Merges location_frame and visitation_frame
# Note we are left with 57 parks as this is all the visitation frame dataset has
vis_and_loc_frame = location_frame.merge(visitation_frame, how='left', left_on='Park_Name', right_on='Park_Name')

In [139]:
# Merges wildlife frame with our visitation and location frame to result in final data.
# Clean up dataset.
database = wildlife_frame.merge(vis_and_loc_frame, how='inner', left_on='Park_Code', right_on='Park_Code')
database = database.drop(['Park_Name_x', 'Category_Sort', 'Taxon_Record_Status', 'Occurrence'], axis=1)
database = database.rename(columns={'Park_Name_y': 'Park_Name', 'Park_Code': 'Park_Code'})
database = database[['Park_Code', 'Park_Name', 'Category', 'Order', 'Family', 'Scientific_Name', \
                      'State', 'Acres', 'Latitude', 'Longitude', 'Visitors']]
print(database.columns)
conn = sqlite3.connect('../data.db')
c = conn.cursor()
database.to_sql('final', conn, if_exists='replace')

Index(['Park_Code', 'Park_Name', 'Category', 'Order', 'Family',
       'Scientific_Name', 'State', 'Acres', 'Latitude', 'Longitude',
       'Visitors'],
      dtype='object')
